In [57]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib
import matplotlib.pyplot as plt

from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.cross_validation import cross_val_score
from scipy.stats import skew

''' Calculates RMSE '''
def rmse_cv(model):
    rmse= np.sqrt(-cross_val_score(model, X_train, y, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)

''' Pre-processing '''
train = pd.read_csv('/Users/anthonydepalatis/Documents/School/EE379K/Lab4/train.csv')
test = pd.read_csv('/Users/anthonydepalatis/Documents/School/EE379K/Lab4/test.csv')
all_data = pd.concat((train.loc[:,'MSSubClass':'SaleCondition'],
                      test.loc[:,'MSSubClass':'SaleCondition']))

# Log transform the classification set
train["SalePrice"] = np.log1p(train["SalePrice"])
prices = pd.DataFrame(train["SalePrice"])

# Log transform skewed numeric features
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index
skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index
all_data[skewed_feats] = np.log1p(all_data[skewed_feats])

# Convert categorical features into dummy variables
all_data = pd.get_dummies(all_data)

# Fill empty cells with the mean of the column
all_data = all_data.fillna(all_data.mean())

# Create matrices for sklearn
X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]
y = train.SalePrice

''' Part 2 '''
# Train a ridge regression model
alphas = [0.01, 0.05, 0.1, 0.5, 1, 2, 4, 8, 10, 15, 30, 45, 60]
model_ridge = [rmse_cv(Ridge(alpha=alpha, solver='svd', tol=0.001)).mean() for alpha in alphas]
model_ridge = pd.Series(model_ridge, index = alphas)
model_ridge.min()

# Train a lasso regression model

0.12733734668670754